In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd 
import sys
import pickle
import itertools
import seaborn as sns
import matplotlib.pyplot as plt

workingdirectory = os.popen('git rev-parse --show-toplevel').read()[:-1]
sys.path.append(workingdirectory)
os.chdir(workingdirectory)

import allensdk.core.json_utilities as ju
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

from mcmodels.core import Mask,ModelData,VoxelModelCache
from mcmodels.core.utils import get_structure_id, get_ordered_summary_structures,get_minorstructures,get_loss_paper
from mcmodels.utils import nonzero_unique, unionize
from mcmodels.core.experiment import get_voxeldata_msvd
from mcmodels.models.crossvalidation import get_best_hyperparameters,get_loss_best_hyp,get_loocv_predictions,get_loss
from mcmodels.core.utils import get_cre_status,get_minorstructure_dictionary,get_leaves_ontologicalorder
from mcmodels.core.utils import get_regionalized_normalized_data
from mcmodels.core.utils import get_connectivity
from mcmodels.core.utils import get_ontological_order_leaf
from mcmodels.core.utils import get_nw_loocv,get_wt_inds
from mcmodels.core.utils import get_countvec

/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
#read data
TOP_DIR = '/Users/samsonkoelle/alleninstitute/mcm_2020/mcm_updated/'
INPUT_JSON = os.path.join(TOP_DIR, 'input_011520.json')
EXPERIMENTS_EXCLUDE_JSON = os.path.join(TOP_DIR, 'experiments_exclude.json')
FILE_DIR = '/Users/samsonkoelle/alleninstitute/mcm_2020/mcm_updated/'
OUTPUT_DIR = os.path.join(FILE_DIR, 'output')

input_data = ju.read(INPUT_JSON)
manifest_file = input_data.get('manifest_file')
manifest_file = os.path.join(TOP_DIR, manifest_file)
experiments_exclude = ju.read(EXPERIMENTS_EXCLUDE_JSON)

#its unclear why the hyperparameters are loaded from the output directory
cache = VoxelModelCache(manifest_file=manifest_file)
major_structures = input_data.get('structures')
major_structure_ids = [get_structure_id(cache, s) for s in major_structures]
data_info = pd.read_excel('/Users/samsonkoelle/alleninstitute/Whole Brain Cre Image Series_curation only.xlsx', 'all datasets curated_070919pull')
data_info.set_index("id", inplace=True)
ontological_order = get_ordered_summary_structures(cache)

mcc = MouseConnectivityCache(manifest_file = '../connectivity/mouse_connectivity_manifest.json')
st = mcc.get_structure_tree()
ai_map = st.get_id_acronym_map()
ia_map = {value: key for key, value in ai_map.items()}

#regionalize voxel model: compare with regional model
#regional parameters
cre = None
eid_set=None
high_res=False
threshold_injection = False

COARSE_STRUCTURE_SET_ID = 2
DEFAULT_STRUCTURE_SET_IDS = tuple([COARSE_STRUCTURE_SET_ID])
tree = cache.get_structure_tree()
default_structures = tree.get_structures_by_set_id(DEFAULT_STRUCTURE_SET_IDS)
default_structure_ids = [st['id'] for st in default_structures if st['id'] != 934]
#cre= True

In [3]:
msvds = {}
#gammas = np.asarray([0.1])
for sid in major_structure_ids:
    print(sid)
    voxel_data = ModelData(cache, sid)
    print(cre)
    experiment_ids = voxel_data.get_experiment_ids(experiments_exclude=experiments_exclude, cre=cre)
    experiment_ids = np.asarray(list(experiment_ids))    
    msvd = get_voxeldata_msvd(cache, sid,experiments_exclude,default_structure_ids,cre)
    #msvd.l2losses, msvd.paperlosses,msvd.normspredict,msvd.normtrue = single_region_cv(msvd, gammas)
    msvds[sid]  = msvd

512
None
703
None
1089
None
1097
None
315
None
313
None
354
None
698
None
771
None
803
None
477
None
549
None


In [5]:

# def get_nw_loocv(msvd, indices, loocv, hyperparameters):

#     if len(indices) > 1:
#         projections = msvd.reg_proj_vcount_norm_renorm
#         centroids = msvd.centroids
#         nreg = projections.shape[1]
#         nexp = projections.shape[0]
#         nhyp = hyperparameters.shape[0]
#         loocv_predictions = np.zeros((nhyp, nexp, nreg))
#         for g in range(nhyp):
#             loocv_predictions[g, indices] = loocv(projections[indices], centroids[indices], hyperparameters[g])
#         return (loocv_predictions)
#     else:
#         return (np.asarray([]))

In [6]:
def get_homo_loocv(msvd, indices, loocv, hyperparameters):

    if len(indices) > 1:
        projections = msvd.reg_proj_vcount_norm_renorm
        injections = msvd.reg_inj_vcount_norm / np.expand_dims(np.linalg.norm(msvd.reg_inj_vcount_norm, axis = 1),1)
        injections[np.where(np.isnan(injections))] = 0.
        
        nreg = projections.shape[1]
        nexp = projections.shape[0]
        nhyp = hyperparameters.shape[0]
        loocv_predictions = np.zeros((nhyp, nexp, nreg))
        for g in range(nhyp):
            loocv_predictions[g, indices] = loocv(projections[indices], injections[indices], *hyperparameters[g])
        return (loocv_predictions)
    else:
        return (np.asarray([]))

In [12]:
from mcmodels.models.crossvalidation import get_loocv_predictions_nnlinear_number_inj_norm

In [13]:
creline = get_cre_status(data_info, msvds)
experiments_minor_structures = get_minorstructure_dictionary(msvds, data_info)
leavves = get_leaves_ontologicalorder(msvd, ontological_order)

# contra_key = msvd.experiments[list(msvd.experiments.keys())[0]].projection_mask.get_key(structure_ids=leavves, hemisphere_id=1)
# ipsi_key = msvd.experiments[list(msvd.experiments.keys())[0]].projection_mask.get_key(structure_ids=leavves, hemisphere_id=2)

key = list(msvd.experiments.keys())[0]
contra_key = msvd.experiments[key].projection_mask.get_key(structure_ids=ontological_order, hemisphere_id=1)
ipsi_key = msvd.experiments[key].projection_mask.get_key(structure_ids=ontological_order, hemisphere_id=2)

msvds = get_regionalized_normalized_data(msvds,cache, ontological_order,ipsi_key,contra_key,experiments_minor_structures)
thres_ncomp = np.asarray([[1e-10,0]])
wt_2ormore = get_wt_inds(creline)

for sid in major_structure_ids:
    print(sid)
    #print(msvds[sid].projections.shape[0], len(wt_2ormore[sid]))
    msvds[sid].loocv_predictions_all = get_homo_loocv(msvds[sid], np.asarray(list(range(msvds[sid].projections.shape[0]))), get_loocv_predictions_nnlinear_number_inj_norm, thres_ncomp)
    msvds[sid].loocv_predictions_wt = get_homo_loocv(msvds[sid], wt_2ormore[sid], get_loocv_predictions_nnlinear_number_inj_norm, thres_ncomp)


512
exp 0
exp 1
exp 2
exp 3
exp 4
exp 5
exp 6
exp 7
exp 8
exp 9
exp 10
exp 11


/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalidation.py:152: RuntimeWarning: invalid value encountered in true_divide
  predictions[i] = pred / np.linalg.norm(pred)


exp 12
exp 13
exp 14
exp 15
exp 16
exp 17
exp 18
exp 19
exp 20
exp 21
exp 22
exp 23
exp 24
exp 25
exp 26
exp 27
exp 28
exp 29
exp 30
exp 31
exp 32
exp 33
exp 34
exp 35
exp 0
exp 1
exp 2
exp 3
exp 4
exp 5
exp 6
exp 7
exp 8
exp 9
exp 10
exp 11
703
exp 0
exp 1


/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalidation.py:152: RuntimeWarning: invalid value encountered in true_divide
  predictions[i] = pred / np.linalg.norm(pred)
/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalidation.py:152: RuntimeWarning: invalid value encountered in true_divide
  predictions[i] = pred / np.linalg.norm(pred)
/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalidation.py:152: RuntimeWarning: invalid value encountered in true_divide
  predictions[i] = pred / np.linalg.norm(pred)


exp 2
exp 3
exp 4
exp 5
exp 6
1089
exp 0
exp 1
exp 2
exp 3
exp 4
exp 5
exp 6
exp 7
exp 8
exp 9
exp 10
exp 11
exp 12
exp 13
exp 14
exp 15
exp 16
exp 17
exp 18
exp 19
exp 20
exp 21
exp 22
exp 23
exp 24
exp 25
exp 26
exp 27
exp 28
exp 29
exp 30
exp 31
exp 32
exp 33
exp 34
exp 35
exp 36
exp 37
exp 38
exp 39
exp 40
exp 41
exp 42
exp 43
exp 44
exp 45
exp 46
exp 47
exp 48
exp 49
exp 50
exp 51
exp 52
exp 53
exp 54
exp 55
exp 56
exp 57
exp 58
exp 59
exp 60
exp 61
exp 62
exp 63
exp 64
exp 65
exp 66
exp 67
exp 68
exp 69
exp 70
exp 71
exp 72
exp 73
exp 74
exp 75
exp 76
exp 77
exp 78
exp 79
exp 80
exp 81
exp 82
exp 83
exp 84
exp 85
exp 86
exp 87
exp 88
exp 89
exp 90
exp 91
exp 92
exp 93
exp 94
exp 95
exp 96
exp 97
exp 98
exp 99
exp 100
exp 101
exp 102
exp 103
exp 104
exp 105
exp 106
exp 107
exp 108
exp 109
exp 110
exp 111
exp 112
exp 113
exp 114
exp 115
exp 116
exp 117
exp 118
exp 119
exp 120
exp 121
exp 0
exp 1
exp 2
exp 3
exp 4
exp 5
exp 6
exp 7
exp 8
exp 9
exp 10
exp 11
exp 12
exp 13
exp 14
exp 

...

/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalidation.py:152: RuntimeWarning: invalid value encountered in true_divide
  predictions[i] = pred / np.linalg.norm(pred)


exp 5
exp 6
exp 7
exp 8
exp 9
exp 10
exp 11
exp 12
exp 13
exp 14
exp 15
exp 16
exp 17
exp 18
exp 19
exp 20
exp 21
exp 22
exp 23
exp 24
exp 25
exp 26
exp 27
exp 28
exp 29
exp 30
exp 31
exp 32
exp 33
exp 34
exp 0
exp 1
exp 2
exp 3
exp 4
exp 5
exp 6
exp 7
exp 8
771
exp 0
exp 1
exp 2


/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalidation.py:152: RuntimeWarning: invalid value encountered in true_divide
  predictions[i] = pred / np.linalg.norm(pred)


exp 3
exp 4
exp 5
exp 6
exp 7
exp 8
exp 9
exp 10
exp 11
exp 12
exp 13
exp 14
exp 15
exp 16
exp 17
exp 18
exp 19
exp 20
exp 21
exp 22
exp 23
exp 24
exp 25
exp 26
exp 27
exp 28
exp 29
exp 30
exp 31
exp 32
exp 0
exp 1
exp 2
exp 3
803
exp 0
exp 1
exp 2
exp 3
exp 4
exp 5
exp 6
exp 7
exp 8
exp 9
exp 10
exp 11
exp 12
exp 13
exp 14
exp 15
exp 16
exp 17
exp 18
exp 19
exp 20
exp 21
exp 22
exp 23
exp 24
exp 25
exp 26
exp 27
exp 28
exp 29
exp 0
exp 1
exp 2
exp 3
477
exp 0
exp 1
exp 2
exp 3
exp 4
exp 5
exp 6
exp 7
exp 8
exp 9
exp 10
exp 11
exp 12
exp 13
exp 14
exp 15
exp 16
exp 17
exp 18
exp 19
exp 20
exp 21
exp 22
exp 23
exp 24
exp 25
exp 26
exp 27
exp 28
exp 29
exp 30
exp 31
exp 32
exp 33
exp 34
exp 35
exp 36
exp 37
exp 38
exp 39
exp 40
exp 41
exp 42
exp 43
exp 44
exp 45
exp 46
exp 47
exp 48
exp 49
exp 50
exp 51
exp 52
exp 53
exp 54
exp 55
exp 56
exp 57
exp 58
exp 59
exp 60
exp 61
exp 62
exp 63
exp 64
exp 65
exp 66
exp 67
exp 68
exp 69
exp 70
exp 71
exp 72
exp 73
exp 74
exp 75
exp 76
exp 77
exp 0

/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalidation.py:152: RuntimeWarning: invalid value encountered in true_divide
  predictions[i] = pred / np.linalg.norm(pred)


In [7]:
from mcmodels.models.crossvalidation import get_loocv_predictions_nnlinear_number_inj

In [14]:

homo_loocv_predictions_all = {}
homo_loocv_predictions_wt = {}
homo_reg_proj_vcount_norm_renorms= {}
for sid in major_structure_ids:
    homo_loocv_predictions_all[sid ] = msvds[sid].loocv_predictions_all
    homo_loocv_predictions_wt[sid ] = msvds[sid].loocv_predictions_wt
    homo_reg_proj_vcount_norm_renorms[sid ] = msvds[sid].reg_proj_vcount_norm_renorm
    

In [15]:

inds_good = {}
for sid in major_structure_ids:
    inds_good[sid] = np.asarray(list(range(msvds[sid].injections.shape[0])))
    
a = [list(range(1)), list(range(1))]
keys = np.asarray(list(itertools.product(*a)))
#keys = np.asarray([list(range(keys.shape[0]))])
#sel_ga_wt = get_best_hyperparameters(losses_wts,keys)



In [16]:

def get_loss(true_dict, prediction_dict, pred_ind=None, true_ind=None, keys=None):
    output = {}
    
    major_structure_ids = list(prediction_dict.keys())
    nms = len(major_structure_ids)
    ngam = prediction_dict[major_structure_ids[0]].shape[0]
    nalph = prediction_dict[major_structure_ids[0]].shape[1]
    for m in range(nms):
        sid = major_structure_ids[m]
        if pred_ind == None:
            # prediction_dict and true_dict will contain predictions for 'bad' experiments with no recorded injection
            # when we have the wild type predictions, the subset is what is good among the wild types
            # so 'true' subsetting is always good, since it is w.r.t. the full injection
            # but prediction needs good w.r.t. wt
            pind = np.asarray(list(range(prediction_dict[sid].shape[1])), dtype=int)
        else:
            pind = pred_ind[sid]
        if true_ind == None:
            tind = np.asarray(list(range(true_dict[sid].shape[0])), dtype=int)
        else:
            tind = true_ind[sid]

        nexp = len(pind)

        output[sid] = np.zeros((keys.shape[0], nexp))

        for j in range(keys.shape[0]):
            output[sid][j] = np.asarray(
                [get_loss_paper(true_dict[sid][tind[i]], prediction_dict[sid][j][pind[i]]) for i in
                 range(nexp)])

    return (output)

In [17]:
losses_all = get_loss(homo_reg_proj_vcount_norm_renorms, homo_loocv_predictions_all,pred_ind = inds_good, true_ind = inds_good,keys = keys)
losses_wts = get_loss(homo_reg_proj_vcount_norm_renorms, homo_loocv_predictions_wt,pred_ind = wt_2ormore, true_ind = wt_2ormore, keys = keys)
losses_allwts = get_loss(homo_reg_proj_vcount_norm_renorms, homo_loocv_predictions_all,pred_ind = wt_2ormore, true_ind = wt_2ormore, keys = keys)


In [18]:

sel_ga_all = get_best_hyperparameters(losses_all,keys)
sel_ga_allwt = sel_ga_all#get_gamma(losses_allwt,keys)

mean_nw_all = get_loss_best_hyp(losses_all, sel_ga_all)
mean_nw_allwt = get_loss_best_hyp(losses_allwts, sel_ga_all)
#mean_nw_wt = get_loss_best_hyp(losses_all, sel_ga_wt)

print(mean_nw_all)
print(mean_nw_allwt)

losses = np.asarray([mean_nw_all, 
           mean_nw_allwt]).transpose()
losses2 = losses[[4,7,2,1,10,9,11,3,5,8,6,0]]
loss =pd.DataFrame(losses2, columns = ['all','allwt'])

0
1
2
3
4
5
6
7
8
9
10
11
[0.73357497 0.85418051 0.33094859 0.2797246  0.42350816 0.48577327
 0.42672229 0.41038508 0.556411   0.58513993 0.65333535 0.37239128]
[0.86530157        nan 0.25243498 0.33765169 0.30536101 0.48097761
 0.65418835 0.28883979 0.49643184 0.40926303 0.43059554 0.29425866]


/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/mcmodels/models/crossvalidation.py:242: RuntimeWarning: Mean of empty slice
  output[m] = np.nanmean(losses[sid][hyps[m], :])


In [21]:
loss.to_clipboard()

In [19]:
name = 'majorsum_homonormed2x_0720'
expfolder = workingdirectory +   '/data/' + name
os.mkdir(expfolder)
loss.to_csv(expfolder + 'losses')

#msvds_save = {}
for sid in major_structure_ids:
    np.save(expfolder +'/'+ str(sid) + 'reg_proj_vcount_norm_renorm',msvds[sid].reg_proj_vcount_norm_renorm)
    np.save(expfolder +'/'+ str(sid) + 'loocv_predictions_all',msvds[sid].loocv_predictions_all)
    np.save(expfolder +'/'+ str(sid) + 'loocv_predictions_wt',msvds[sid].loocv_predictions_wt)
    
with open(expfolder +'/'+ 'wt_2ormore_base.pickle', 'wb') as handle:
    pickle.dump(wt_2ormore, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('data/info/leafs.pickle', 'rb') as handle:
    leafs = pickle.load(handle)
    
#ontological_order_leaf = get_ontological_order_leaf(leafs, ontological_order, st)
#source_exp_countvec, source_exp_countvec_wt = get_countvec(ontological_order, ia_map, creline, experiments_minor_structures)
#hyperparameters = gammas[sel_ga_all]
#leavves v ontological_order_leaf
#leavves is the leaves associated with ontological_order
#ontological_order_leaf is the leaves that are actually present
#leafs is the dictionary with the leaves of the experiments
#df = get_connectivity(msvds, cache, ia_map, hyperparameters, ontological_order_leaf, ontological_order, leafs, creline,experiments_minor_structures, ipsi_key, contra_key)

In [10]:
name = 'majorsum_homo_0720'
expfolder = workingdirectory +   '/data/' + name

homo_loocv_predictions_all = {}
homo_loocv_predictions_wt = {}
homo_reg_proj_vcount_norm_renorms= {}
for sid in major_structure_ids:
    homo_loocv_predictions_all[sid ] = np.load(expfolder +'/'+ str(sid) + 'loocv_predictions_all')
    homo_loocv_predictions_wt[sid ] = np.load(expfolder +'/'+ str(sid) + 'loocv_predictions_wt')
    homo_reg_proj_vcount_norm_renorms[sid ] = msvds[sid].reg_proj_vcount_norm_renorm
    

FileNotFoundError: [Errno 2] No such file or directory: '/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/data/majorsum_homo_0720/512loocv_predictions_all'

In [11]:
# hyperparameters =thres_ncomp
# indices = np.asarray(list(range(msvds[sid].projections.shape[0])))
# msvd = msvds[sid]

In [16]:
#     if len(indices) > 1:
#         projections = msvd.reg_proj_vcount_norm_renorm
#         injections = msvd.reg_inj_vcount_norm / np.expand_dims(np.linalg.norm(msvd.reg_inj_vcount_norm, axis = 1),1)
#         injections[np.where(np.isnan(injections))] = 0.
        
#         nreg = projections.shape[1]
#         nexp = projections.shape[0]
#         nhyp = hyperparameters.shape[0]
#         loocv_predictions = np.zeros((nhyp, nexp, nreg))
#         for g in range(nhyp):
#             loocv_predictions[g, indices] = get_loocv_predictions_nnlinear_number_inj(projections[indices], injections[indices], *hyperparameters[g])
#         #return (loocv_predictions)
#     else:
#         2+2
#         #return (np.asarray([]))

In [15]:
hyperparameters[g]

array([1.e-10, 0.e+00])

In [20]:
get_loocv_predictions_nnlinear_number_inj(projections[indices], injections[indices], *hyperparameters[g])

array([[8.37970339e-03, 2.98649993e-05, 1.32495100e-02, ...,
        7.23178018e-05, 1.21523750e-04, 9.72360067e-05],
       [1.94261614e-02, 1.84335659e-05, 4.46369778e-03, ...,
        0.00000000e+00, 8.17640903e-05, 1.34749070e-03],
       [2.73950435e-02, 1.19056692e-02, 5.60228727e-05, ...,
        5.72578028e-06, 1.03043303e-05, 3.50847782e-04],
       ...,
       [5.06741181e-03, 2.41173602e-05, 1.96433789e-03, ...,
        0.00000000e+00, 0.00000000e+00, 2.05604068e-04],
       [3.45455036e-02, 1.45014897e-02, 4.23544261e-05, ...,
        6.50257243e-06, 1.54651425e-05, 5.33802784e-04],
       [9.55329233e-05, 4.14985407e-05, 1.31462144e-07, ...,
        1.79815256e-08, 4.32830056e-08, 1.48560673e-06]])

In [19]:
*hyperparameters[g]

SyntaxError: can't use starred expression here (<ipython-input-19-870c7af242e9>, line 4)

In [21]:
get_loocv_predictions_nnlinear_number_inj(projections[indices], injections[indices], hyperparameters[g][0], hyperparameters[g][1])

array([[8.37970339e-03, 2.98649993e-05, 1.32495100e-02, ...,
        7.23178018e-05, 1.21523750e-04, 9.72360067e-05],
       [1.94261614e-02, 1.84335659e-05, 4.46369778e-03, ...,
        0.00000000e+00, 8.17640903e-05, 1.34749070e-03],
       [2.73950435e-02, 1.19056692e-02, 5.60228727e-05, ...,
        5.72578028e-06, 1.03043303e-05, 3.50847782e-04],
       ...,
       [5.06741181e-03, 2.41173602e-05, 1.96433789e-03, ...,
        0.00000000e+00, 0.00000000e+00, 2.05604068e-04],
       [3.45455036e-02, 1.45014897e-02, 4.23544261e-05, ...,
        6.50257243e-06, 1.54651425e-05, 5.33802784e-04],
       [9.55329233e-05, 4.14985407e-05, 1.31462144e-07, ...,
        1.79815256e-08, 4.32830056e-08, 1.48560673e-06]])